In [1]:
#!/usr/bin/python
import re
import itertools

In [2]:
profiles = {}
pattern = re.compile(r"\s*\d+")
with open("data/ss1_16_indepTestSet_groups5_4.profile") as prof_file:
    lines = prof_file.readlines()
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.startswith(">"):
            header = line[1:]
            if(header.find("#") >= 0): 
                header = header.split("#")[1]
            prof = []
            i+=5
        elif pattern.match(line):
            prof.append(line)
            i+=1
        else: 
            profiles[header] = prof
            i+=6      

In [3]:
def get_profile(header, start, stop, mname): 
    prof = [
        """    
Last position-specific scoring matrix computed, weighted observed percentages rounded down, information per position, and relative weight of gapless real matches to pseudocounts
           A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V   A   R   N   D   C   Q   E   G   H   I   L   K   M   F   P   S   T   W   Y   V"""
    ]
    prof.extend(profiles[header][start:stop])
    return prof

In [4]:
daten = open("data/ss1_16_indepTestSet_groups5_6.txt").readlines()
lens = [25, 30]
# lens = [25]
# method_ids = [0]
method_ids = [0, 1, 2]
method_patterns = ['\s\d+\s(\w)\s\d.\d\d\s([D-])', 
                   '\s\d+\s(\w)\s\d.\d\d\s[D-]\s\d.\d\d\s([D-])',
                   '\s\d+\s(\w)\s\d.\d\d\s[D-]\s\d.\d\d\s[D-]\s\d.\d\d\s([D-])']
method_names = ["norsnet", "bval", "ucon"]

for rlen, mid in itertools.product(lens, method_ids):
    pattern = method_patterns[mid]
    mname = method_names[mid]
    !mkdir -p data/testset{rlen}.{mname}/possitiv  data/testset{rlen}.{mname}/negativ
    i=0
    countO = 1
    countD = 1
    while i<len(daten) :
        if daten[i].startswith(">"):
            pos = 0
            countO = countD = 1
            header = daten[i][1:].strip().split("#")[1]
            i=i+3
            zeile = re.search(pattern,daten[i])
            now = zeile.group(2)
            sequenz = [zeile.group(1)]
            i= i+1
        elif daten[i].startswith(" "):
            zeile = re.search(pattern,daten[i])
            pos += 1
            if now == zeile.group(2) and countD <rlen and countO < rlen:
                sequenz.append(zeile.group(1))
                if now=='D':
                    countD = countD+1
                    countO = 1
                else:
                    countO = countO+1
                    countD = 1
            elif countD == rlen:                
                sequenz = ''.join(sequenz)
                assert len(sequenz) == rlen, (countD, len(sequenz), sequenz)
                basename = "data/testset{}.{}/possitiv/region_1_{}_{}".format(rlen,mname,header,pos-rlen)
                newDisorder= open(basename +'.fa','w')
                newDisorderMat = open(basename + ".fa.blastPsiMat", 'w')
                newDisorder.write(">" + header+ " " + mname + "\n"+ sequenz)
                newDisorderMat.write("\n".join(get_profile(header, pos-rlen, pos, mname)) + "\n")
                newDisorderMat.close()
                newDisorder.close()
                sequenz=[zeile.group(1)]
                countD =1
                now = zeile.group(2)
            elif countO == rlen:
                sequenz = ''.join(sequenz)
                assert len(sequenz) == rlen, (countO, len(sequenz), sequenz)
                basename = "data/testset{}.{}/negativ/region_1_{}_{}".format(rlen,mname,header,pos-rlen)
                newOrder= open(basename + '.fa','w')
                newOrderMat = open(basename + ".fa.blastPsiMat", 'w')
                newOrderMat.write("\n".join(get_profile(header, pos-rlen, pos, mname)) + "\n")
                newOrderMat.close()
                newOrder.write(">" + header+ " " + mname + "\n"+ sequenz)
                newOrder.close()
                countO=1
                sequenz = [zeile.group(1)]
                now = zeile.group(2)
            else:
                sequenz= [zeile.group(1)]
                countO=1
                countD=1
                now = zeile.group(2)
            i=i+1
        else:
            i=i+1
